<a href="https://colab.research.google.com/github/MhmdSheref/MEDSEG/blob/master/MEDSEG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Setup
Downloading required pip packages and the preview dataset

In [ ]:
!pip install TotalSegmentator gdown "pyvista[jupyter]" "jupyterlab" pynrrd

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [ ]:
!gdown "https://drive.google.com/uc?export=download&id=1mgwuCpvTc3INnqGHiXhxJsbEzZPasplx" -O CT_subset_big.zip

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1mgwuCpvTc3INnqGHiXhxJsbEzZPasplx
From (redirected): https://drive.google.com/uc?export=download&id=1mgwuCpvTc3INnqGHiXhxJsbEzZPasplx&confirm=t&uuid=78cd4be8-a421-44d9-8489-17d1b8721c91
To: /content/CT_subset_big.zip
100% 452M/452M [00:03<00:00, 131MB/s]


In [ ]:
!unzip CT_subset_big.zip -d CT_Set

Archive:  CT_subset_big.zip
replace CT_Set/s0000/segmentations/iliopsoas_left.nii.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Segmentation
Select one of the included preview CT scans as well as the AI model to use:

In [ ]:
# @title
import ipywidgets as widgets
from IPython.display import display
import os

# Get list of subject folders in CT_Set
ct_set_path = "CT_Set"
subject_folders = [f.path for f in os.scandir(ct_set_path) if f.is_dir() and f.name.startswith('s')]
subject_folders.sort()

# Create a dropdown widget to select a subject folder
subject_selector = widgets.Dropdown(
    options=subject_folders,
    description='Select Subject Folder:',
    disabled=False,
    style = {'description_width': '150px'}
)

# Define the available models
available_models = ['totalSegmentator', 'Nvidia vista-3d'] # Add other models as needed

# Create a dropdown widget for model selection
model_selector = widgets.Dropdown(
    options=available_models,
    description='Select AI Model:',
    disabled=False,
    style = {'description_width': '150px'}
)

display(subject_selector, model_selector)

Dropdown(description='Select Subject Folder:', options=('CT_Set/s0000', 'CT_Set/s0001', 'CT_Set/s0002', 'CT_Se…

Dropdown(description='Select AI Model:', options=('totalSegmentator', 'Nvidia vista-3d'), style=DescriptionSty…

### Visualize the selected CT scan

In [ ]:
# @title
import nibabel as nib
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive, Output
import os
from IPython.display import display

selected_folder = subject_selector.value

# Load the NIfTI file
file_path = os.path.join(selected_folder, "ct.nii.gz")
img = nib.load(file_path)
data = img.get_fdata()

# Get dimensions
n_slices_axial = data.shape[2]
n_slices_coronal = data.shape[1]
n_slices_sagittal = data.shape[0]

# Outputs for manual layout
out_axial = Output()
out_coronal = Output()
out_sagittal = Output()

# Define slice viewers for each axis
def view_axial_slice(slice_idx):
    with out_axial:
        out_axial.clear_output(wait=True)
        plt.figure(figsize=(4, 4))
        plt.imshow(data[:, :, slice_idx].T, cmap="gray", origin="lower")
        plt.title(f"Axial {slice_idx}")
        plt.axis("off")
        plt.show()

def view_coronal_slice(slice_idx):
    with out_coronal:
        out_coronal.clear_output(wait=True)
        plt.figure(figsize=(4, 4))
        plt.imshow(data[:, slice_idx, :].T, cmap="gray", origin="lower")
        plt.title(f"Coronal {slice_idx}")
        plt.axis("off")
        plt.show()

def view_sagittal_slice(slice_idx):
    with out_sagittal:
        out_sagittal.clear_output(wait=True)
        plt.figure(figsize=(4, 4))
        plt.imshow(data[slice_idx, :, :].T, cmap="gray", origin="lower")
        plt.title(f"Sagittal {slice_idx}")
        plt.axis("off")
        plt.show()

# Sliders
axial_slider = widgets.IntSlider(min=0, max=n_slices_axial - 1, value=n_slices_axial // 2, description="Axial")
coronal_slider = widgets.IntSlider(min=0, max=n_slices_coronal - 1, value=n_slices_coronal // 2, description="Coronal")
sagittal_slider = widgets.IntSlider(min=0, max=n_slices_sagittal - 1, value=n_slices_sagittal // 2, description="Sagittal")

# Link sliders to update functions
widgets.interactive_output(view_axial_slice, {'slice_idx': axial_slider})
widgets.interactive_output(view_coronal_slice, {'slice_idx': coronal_slider})
widgets.interactive_output(view_sagittal_slice, {'slice_idx': sagittal_slider})

# Show initial slices
view_axial_slice(axial_slider.value)
view_coronal_slice(coronal_slider.value)
view_sagittal_slice(sagittal_slider.value)

# Layout: three images in a row, sliders below
display(
    widgets.VBox([
        widgets.HBox([out_axial, out_coronal, out_sagittal]),
        widgets.HBox([axial_slider, coronal_slider, sagittal_slider])
    ])
)

### Run inference/API call and process output

In [ ]:
from totalsegmentator.python_api import totalsegmentator
import requests
from google.colab import userdata
import os
import shutil
import zipfile
import io

input_file = os.path.join(selected_folder, "ct.nii.gz")

output_dir = "output_segments"

# Delete the output directory if it exists
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

# Create the output directory
os.makedirs(output_dir, exist_ok=True)


match model_selector.value:
    case 'totalSegmentator':
        totalsegmentator(input_file, output_dir)
    case 'Nvidia vista-3d':
        temp_download_dir = os.path.join(output_dir, "temp_download") # Use a dedicated temp dir for download

        invoke_url = "https://health.api.nvidia.com/v1/medicalimaging/nvidia/vista-3d"
        headers = {
            "authorization": f"Bearer {userdata.get('NVIDIA_API_KEY')}" # Get API key from Colab secrets
        }
        payload = {
            "output": {
                "extension": ".nrrd", # Request NRRD output
                "dtype": "uint8"
            },
            "image": "https://raw.githubusercontent.com/NVIDIA/ai-assisted-annotation-client/master/py_client/test-data/image.nii.gz"
        }
        # re-use connections
        session = requests.Session()
        response = session.post(invoke_url, headers=headers, json=payload)
        response.raise_for_status()

        # Create the temporary download directory
        os.makedirs(temp_download_dir, exist_ok=True)

        try:
            z = zipfile.ZipFile(io.BytesIO(response.content))
            z.extractall(temp_download_dir)
            file_list = os.listdir(temp_download_dir)
            for filename in file_list:
                filepath = os.path.join(temp_download_dir, filename)
                # Check for .nrrd files and move them to output_dir, renaming to "result.nrrd"
                if os.path.isfile(filepath) and filename.endswith(".nrrd"):
                    destination_path = os.path.join(output_dir, "result.nrrd")
                    shutil.move(filepath, destination_path) # Save as "result.nrrd", overwriting if exists
                    print(f"Saved {filename} as {os.path.basename(destination_path)}")
        finally:
            # Clean up the temporary directory
            if os.path.exists(temp_download_dir):
                shutil.rmtree(temp_download_dir)


    case 'case3':
        pass


If you use this tool please cite: https://pubs.rsna.org/doi/10.1148/ryai.230024

Resampling...
  Resampled in 9.36s
Predicting part 1 of 5 ...


100%|██████████| 30/30 [00:03<00:00,  9.54it/s]


Predicting part 2 of 5 ...


100%|██████████| 30/30 [00:03<00:00,  9.56it/s]


Predicting part 3 of 5 ...


...

Predicting part 4 of 5 ...


100%|██████████| 30/30 [00:03<00:00,  9.42it/s]


Predicting part 5 of 5 ...


100%|██████████| 30/30 [00:03<00:00,  9.55it/s]


  Predicted in 112.94s
Resampling...
Saving segmentations...
  Saved in 16.27s


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import nibabel as nib
import numpy as np
import nrrd

# Assume output_dir is defined in a previous cell or define it here if needed
# For example, if output_dir is not defined:
# output_dir = "output_segments"

valid_segmentation_names = []
# Define a mapping from class IDs to organ names (You need to fill this in based on your model)
# Example: class_id_to_organ_name = {1: "liver", 2: "kidney_right", ...}
class_id_to_organ_name = {
    1: "liver",
    3: "spleen",
    4: "pancreas",
    5: "kidney_right",
    6: "aorta",
    7: "inferior_vena_cava",
    8: "adrenal_gland_right",
    9: "adrenal_gland_left",
    10: "gallbladder",
    11: "esophagus",
    12: "stomach",
    13: "duodenum",
    14: "kidney_left",
    15: "urinary_bladder",
    17: "portal_vein_and_splenic_vein",
    19: "small_bowel",
    22: "brain",
    23: "lung tumor", # No direct mapping in TotalSegmentator list
    24: "pancreatic tumor", # No direct mapping in TotalSegmentator list
    25: "hepatic vessel", # No direct mapping in TotalSegmentator list
    26: "hepatic tumor", # No direct mapping in TotalSegmentator list
    27: "colon cancer primaries", # No direct mapping in TotalSegmentator list
    28: "lung_upper_lobe_left",
    29: "lung_lower_lobe_left",
    30: "lung_upper_lobe_right",
    31: "lung_middle_lobe_right",
    32: "lung_lower_lobe_right",
    33: "vertebrae_L5",
    34: "vertebrae_L4",
    35: "vertebrae_L3",
    36: "vertebrae_L2",
    37: "vertebrae_L1",
    38: "vertebrae_T12",
    39: "vertebrae_T11",
    40: "vertebrae_T10",
    41: "vertebrae_T9",
    42: "vertebrae_T8",
    43: "vertebrae_T7",
    44: "vertebrae_T6",
    45: "vertebrae_T5",
    46: "vertebrae_T4",
    47: "vertebrae_T3",
    48: "vertebrae_T2",
    49: "vertebrae_T1",
    50: "vertebrae_C7",
    51: "vertebrae_C6",
    52: "vertebrae_C5",
    53: "vertebrae_C4",
    54: "vertebrae_C3",
    55: "vertebrae_C2",
    56: "vertebrae_C1",
    57: "trachea",
    58: "iliac_artery_left",
    59: "iliac_artery_right",
    60: "iliac_vena_left",
    61: "iliac_vena_right",
    62: "colon",
    63: "rib_left_1",
    64: "rib_left_2",
    65: "rib_left_3",
    66: "rib_left_4",
    67: "rib_left_5",
    68: "rib_left_6",
    69: "rib_left_7",
    70: "rib_left_8",
    71: "rib_left_9",
    72: "rib_left_10",
    73: "rib_left_11",
    74: "rib_left_12",
    75: "rib_right_1",
    76: "rib_right_2",
    77: "rib_right_3",
    78: "rib_right_4",
    79: "rib_right_5",
    80: "rib_right_6",
    81: "rib_right_7",
    82: "rib_right_8",
    83: "rib_right_9",
    84: "rib_right_10",
    85: "rib_right_11",
    86: "rib_right_12",
    87: "humerus_left",
    88: "humerus_right",
    89: "scapula_left",
    90: "scapula_right",
    91: "clavicula_left",
    92: "clavicula_right",
    93: "femur_left",
    94: "femur_right",
    95: "hip_left",
    96: "hip_right",
    97: "sacrum",
    98: "gluteus_maximus_left",
    99: "gluteus_maximus_right",
    100: "gluteus_medius_left",
    101: "gluteus_medius_right",
    102: "gluteus_minimus_left",
    103: "gluteus_minimus_right",
    104: "autochthon_left",
    105: "autochthon_right",
    106: "iliopsoas_left",
    107: "iliopsoas_right",
    108: "atrial_appendage_left",
    109: "brachiocephalic_trunk",
    110: "brachiocephalic_vein_left",
    111: "brachiocephalic_vein_right",
    112: "common_carotid_artery_left",
    113: "common_carotid_artery_right",
    114: "costal_cartilages",
    115: "heart",
    116: "kidney_cyst_left",
    117: "kidney_cyst_right",
    118: "prostate",
    119: "pulmonary_vein",
    120: "skull",
    121: "spinal_cord",
    122: "sternum",
    123: "subclavian_artery_left",
    124: "subclavian_artery_right",
    125: "superior_vena_cava",
    126: "thyroid_gland",
    127: "vertebrae_S1",
    128: "bone lesion", # No direct mapping in TotalSegmentator list
    132: "airway" # No direct mapping in TotalSegmentator list
}


# Determine the expected file extension based on the selected model
selected_model = model_selector.value
if selected_model == 'Nvidia vista-3d':
    input_extension = ".nrrd"
else:
    input_extension = ".nii.gz"


# Get list of all segmentation files with the expected input extension
# When using Nvidia vista-3d, we expect only one file named "result.nrrd"
if selected_model == 'Nvidia vista-3d':
    all_segmentation_files = ["result.nrrd"]
else:
    all_segmentation_files = [f for f in os.listdir(output_dir) if f.endswith(input_extension)]

# Filter segmentation files based on valid geometry and convert if necessary
for file_name in all_segmentation_files:
    file_path = os.path.join(output_dir, file_name)
    base_name = file_name.replace(input_extension, "")

    # Skip if the file doesn't exist (relevant for "result.nrrd" if download failed)
    if not os.path.exists(file_path):
        print(f"Skipping {file_name}: File not found.")
        continue

    try:
        if input_extension == ".nii.gz":
            img = nib.load(file_path)
            data = img.get_fdata()

            # If the input was NIfTI and not split, add its base name to valid names
            # Add to valid_segmentation_names only if it contains at least 5 non-zero voxels
            if np.count_nonzero(data) >= 5: # Only add if it contains at least 5 non-zero voxels
               valid_segmentation_names.append(base_name)
            else:
                print(f"Skipping {file_name}: Data has less than 5 non-zero voxels.")


        elif input_extension == ".nrrd":
            print(f"Processing NRRD file: {file_name}")
            data = nrrd.read(file_path)

            # If the data contains multiple unique non-zero values, split it
            unique_values = np.unique(data[data != 0])
            if len(unique_values) > 0: # Handle cases with only background
                print(f"Splitting {file_name} into multiple organ files...")
                for organ_id in unique_values:
                    organ_mask = (data == organ_id).astype(data.dtype)
                    # Get organ name from mapping, use class ID if not found
                    organ_name = class_id_to_organ_name.get(int(organ_id), f"class_{int(organ_id)}")
                    # Create a new NIfTI image for each organ
                    organ_img = nib.Nifti1Image(organ_mask, np.eye(4)) # Using identity affine matrix
                    organ_output_path = os.path.join(output_dir, f"{base_name}_{organ_name}.nii.gz")
                    nib.save(organ_img, organ_output_path)
                    # Add to valid_segmentation_names only if the organ mask has at least 5 non-zero voxels
                    if np.count_nonzero(organ_mask) >= 5:
                         valid_segmentation_names.append(f"{base_name}_{organ_name}")
                         print(f"  Saved: {os.path.basename(organ_output_path)}")
                    else:
                         print(f"  Skipping saving {os.path.basename(organ_output_path)}: Mask has less than 5 non-zero voxels.")

    except Exception as e:
        print(f"Could not load, process, or validate {file_name}: {e}")
        continue

# Sort the valid segmentation names alphabetically
valid_segmentation_names.sort()

Skipping brain.nii.gz: Data has less than 5 non-zero voxels.
Skipping vertebrae_C3.nii.gz: Data has less than 5 non-zero voxels.
Skipping prostate.nii.gz: Data has less than 5 non-zero voxels.
Skipping vertebrae_C2.nii.gz: Data has less than 5 non-zero voxels.
Skipping vertebrae_C6_liver.nii.gz: Data has less than 5 non-zero voxels.
Skipping vertebrae_C1.nii.gz: Data has less than 5 non-zero voxels.
Skipping kidney_cyst_left.nii.gz: Data has less than 5 non-zero voxels.
Skipping kidney_cyst_right.nii.gz: Data has less than 5 non-zero voxels.
Skipping vertebrae_C4.nii.gz: Data has less than 5 non-zero voxels.
Skipping vertebrae_C6.nii.gz: Data has less than 5 non-zero voxels.
Skipping vertebrae_C5.nii.gz: Data has less than 5 non-zero voxels.


# Visualizer
Select the organs to be shown and then run the next cell to visualize them in 3d

In [ ]:
# @title
# Create a dropdown widget to select organs
organ_selector = widgets.SelectMultiple(
    options=valid_segmentation_names,
    description='Select Organs:',
    disabled=False,
    style = {'description_width': 'initial'}
)

display(organ_selector)

SelectMultiple(description='Select Organs:', options=('adrenal_gland_left', 'adrenal_gland_left_liver', 'adren…

In [ ]:
# @title
import nibabel as nib
import pyvista as pv
from skimage import measure
import numpy as np
from scipy.ndimage import gaussian_filter
import colorsys


# This cell should be run after selecting organs in the previous cell

# Assume output_dir is defined in a previous cell or define it here if needed
# Assume organ_selector and its value are available from the previous cell

loaded_meshes = {}
plotter = pv.Plotter(notebook=True)
organ_controls = {}

# Load selected organ masks and create meshes
selected_organs = organ_selector.value

if not selected_organs:
    print("No organs selected. Please select organs in the previous cell.")
else:
    num_organs = len(selected_organs)
    for i, organ_name in enumerate(selected_organs):
        mask_file = os.path.join(output_dir, f"{organ_name}.nii.gz")
        if os.path.exists(mask_file):
            mask = nib.load(mask_file).get_fdata()
            mask_smoothed = gaussian_filter(mask.astype(float), sigma=1.0)
            verts, faces, _, _ = measure.marching_cubes(mask_smoothed, level=0.5, step_size=1)
            faces = np.hstack((np.ones((faces.shape[0], 1)) * 3, faces)).flatten().astype(np.int64)
            mesh = pv.PolyData(verts, faces)
            loaded_meshes[organ_name] = mesh

            # Calculate initial color using HSV
            hue = i / num_organs  # Distribute hues evenly
            rgb_color = colorsys.hsv_to_rgb(hue, 0.8, 0.8)  # Use a fixed saturation and value
            hex_color = '#%02x%02x%02x' % (int(rgb_color[0]*255), int(rgb_color[1]*255), int(rgb_color[2]*255))

            # Add mesh to plotter and store the actor
            actor = plotter.add_mesh(mesh, color=hex_color, opacity=0.6, name=organ_name)

            # Create controls for each organ
            color_picker = widgets.ColorPicker(concise=False, description='Color:', value=hex_color, disabled=False)
            opacity_slider = widgets.FloatSlider(value=0.6, min=0.0, max=1.0, step=0.05, description='Opacity:', disabled=False, continuous_update=True, orientation='horizontal', readout=True, readout_format='.2f')
            visibility_checkbox = widgets.Checkbox(value=True, description='Visible:', disabled=False)
            render_button = widgets.Button(description="Render Plot")

            # Store controls and actor
            organ_controls[organ_name] = {
                'color_picker': color_picker,
                'opacity_slider': opacity_slider,
                'visibility_checkbox': visibility_checkbox,
                'actor': actor
            }

            # Link controls to actor properties - these will update the actor directly but not re-render
            def update_color(change, actor=actor):
                actor.prop.color = change['new']

            def update_opacity(change, actor=actor):
                actor.prop.opacity = change['new']

            def update_visibility(change, actor=actor):
                actor.SetVisibility(change['new'])


            color_picker.observe(update_color, names='value')
            opacity_slider.observe(update_opacity, names='value')
            visibility_checkbox.observe(update_visibility, names='value')

    # Display controls and plot

    def render(b=None): # Added b=None to accept button click event
      clear_output()
      for organ_name, controls in organ_controls.items():
          print(f"Controls for {organ_name}:")
          display(widgets.HBox([controls['color_picker'], controls['opacity_slider'], controls['visibility_checkbox']]))
      display(render_button)
      plotter.show(jupyter_backend='html')

      render_button.on_click(render)



    # Initial display of controls and plot
    render()

Controls for lung_lower_lobe_left:


Controls for lung_lower_lobe_left_liver:


Controls for lung_lower_lobe_right:


Controls for lung_lower_lobe_right_liver:


Controls for lung_middle_lobe_right:


...

Controls for lung_upper_lobe_left_liver:


Controls for lung_upper_lobe_right:


Controls for lung_upper_lobe_right_liver:


Button(description='Render Plot', style=ButtonStyle())

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

# Evaluation
Compare the model's predictions to professional predictions by loading both sets of segmentation masks for a selected subject.

Chosen metrics for comparing segmentation masks:
 1. Dice Similarity Coefficient (Dice): Measures the overlap between two segmentation masks.
    > Formula: **Dice = (2 * |A intersect B|) / (|A| + |B|)** Where A and B are the two segmentation masks.\
    Range: 0 (no overlap) to 1 (perfect overlap). Higher is better.

 2. Jaccard Index (IoU - Intersection over Union): Another measure of overlap, similar to Dice.
    > Formula: **Jaccard = |A intersect B| / |A union B|**\
      Range: 0 (no overlap) to 1 (perfect overlap). Higher is better.

 3. Hausdorff Distance: Measures the maximum distance between points in the boundaries of the two segmentation masks.
    > Formula: **Hausdorff(A, B) = max(h(A, B), h(B, A)) where h(A, B) = max_{a in A} min_{b in B} distance(a, b)**\
      Lower is better, as it indicates the boundaries are closer.

Loading the relavant professional segmentations:

### Calculating metrics

In [ ]:
# @title
# Construct the file path to the professional segmentation directory
professional_segmentation_dir = os.path.join(selected_folder, "segmentations")

# List all .nii.gz files within the professional segmentation directory
all_professional_files = [f for f in os.listdir(professional_segmentation_dir) if f.endswith(".nii.gz")]

# Filter to include only the files corresponding to the selected_organs
professional_segmentation_files = [f for f in all_professional_files if f.replace(".nii.gz", "") in selected_organs]

# Create an empty dictionary to store professional masks
professional_masks = {}

# Iterate through the filtered list and load the masks
for file_name in professional_segmentation_files:
    file_path = os.path.join(professional_segmentation_dir, file_name)
    try:
        img = nib.load(file_path)
        data = img.get_fdata()
        organ_name = file_name.replace(".nii.gz", "")
        professional_masks[organ_name] = data
    except Exception as e:
        print(f"Could not load professional mask {file_name}: {e}")
        continue

print(f"Loaded {len(professional_masks)} professional segmentation masks.")

Loaded 5 professional segmentation masks.


Loading the predicted segmentations:

In [ ]:
import os
import nibabel as nib
import numpy as np

# Verify that the output_dir variable exists and contains the segmentation masks
if not os.path.isdir(output_dir):
    print(f"Error: Output directory '{output_dir}' not found.")
else:
    # Create an empty dictionary to store the model's segmentation masks
    model_masks = {}

    # Iterate through the selected_organs list and load the model's masks
    for organ_name in selected_organs:
        mask_file = os.path.join(output_dir, f"{organ_name}.nii.gz")

        # Check if the file exists
        if os.path.exists(mask_file):
            try:
                # Load the NIfTI image and get the data
                img = nib.load(mask_file)
                data = img.get_fdata()

                # Store the NumPy array in the model_masks dictionary
                model_masks[organ_name] = data
            except Exception as e:
                print(f"Could not load model mask {mask_file}: {e}")
        else:
            print(f"Warning: Model segmentation for '{organ_name}' not found at '{mask_file}'.")

    print(f"Loaded {len(model_masks)} model segmentation masks.")

Loaded 10 model segmentation masks.


Calculating the 3 metrics

In [ ]:
from scipy.spatial.distance import directed_hausdorff
import numpy as np

def calculate_metrics(mask1, mask2):
    """
    Calculates Dice coefficient, Jaccard index, and Hausdorff distance between two binary masks.

    Args:
        mask1 (np.ndarray): The first binary mask.
        mask2 (np.ndarray): The second binary mask.

    Returns:
        tuple: A tuple containing the Dice coefficient, Jaccard index, and Hausdorff distance.
               Returns (0.0, 0.0, np.inf) if either mask is empty for Dice and Jaccard,
               and np.inf for Hausdorff distance if either mask is empty.
    """
    mask1_flat = mask1.flatten()
    mask2_flat = mask2.flatten()

    # Dice Coefficient
    intersection = np.sum(mask1_flat * mask2_flat)
    sum_masks = np.sum(mask1_flat) + np.sum(mask2_flat)
    dice = (2.0 * intersection) / sum_masks if sum_masks else 0.0

    # Jaccard Index
    union = np.sum(mask1_flat + mask2_flat - mask1_flat * mask2_flat)
    jaccard = intersection / union if union else 0.0

    # Hausdorff Distance
    coords1 = np.argwhere(mask1)
    coords2 = np.argwhere(mask2)
    hausdorff_distance = np.inf
    if coords1.shape[0] > 0 and coords2.shape[0] > 0:
        # Calculate directed Hausdorff distance in both directions and take the maximum
        h1 = directed_hausdorff(coords1, coords2)[0]
        h2 = directed_hausdorff(coords2, coords1)[0]
        hausdorff_distance = max(h1, h2)
    elif coords1.shape[0] > 0 or coords2.shape[0] > 0:
        # If one mask is empty and the other is not, Hausdorff distance is infinite
         hausdorff_distance = np.inf


    return dice, jaccard, hausdorff_distance

# Store calculated metrics
comparison_results = {}

# Iterate through selected organs and calculate metrics
for organ_name in selected_organs:
    model_mask = model_masks.get(organ_name)
    professional_mask = professional_masks.get(organ_name)

    if model_mask is None:
        print(f"Skipping metrics for '{organ_name}': Model mask not found.")
        continue
    if professional_mask is None:
        print(f"Skipping metrics for '{organ_name}': Professional mask not found.")
        continue

    dice, jaccard, hausdorff = calculate_metrics(model_mask, professional_mask)
    comparison_results[organ_name] = {
        'dice': dice,
        'jaccard': jaccard,
        'hausdorff': hausdorff
    }
    print(f"Metrics for {organ_name}: Dice={dice:.4f}, Jaccard={jaccard:.4f}, Hausdorff={hausdorff:.4f}")

Metrics for lung_lower_lobe_left: Dice=0.9908, Jaccard=0.9817, Hausdorff=7.1414
Skipping metrics for 'lung_lower_lobe_left_liver': Professional mask not found.
Metrics for lung_lower_lobe_right: Dice=0.9903, Jaccard=0.9809, Hausdorff=2.8284
Skipping metrics for 'lung_lower_lobe_right_liver': Professional mask not found.
Metrics for lung_middle_lobe_right: Dice=0.9457, Jaccard=0.8970, Hausdorff=41.7852
Skipping metrics for 'lung_middle_lobe_right_liver': Professional mask not found.
Metrics for lung_upper_lobe_left: Dice=0.9950, Jaccard=0.9900, Hausdorff=3.6056
Skipping metrics for 'lung_upper_lobe_left_liver': Professional mask not found.
Metrics for lung_upper_lobe_right: Dice=0.9829, Jaccard=0.9664, Hausdorff=32.7109
Skipping metrics for 'lung_upper_lobe_right_liver': Professional mask not found.


### Results

In [ ]:
# @title
import pandas as pd
import numpy as np
from IPython.display import display

# Initialize a dictionary to store results if it doesn't exist
if 'all_comparison_results' not in globals():
    all_comparison_results = {}

# Create a pandas DataFrame from the current comparison_results dictionary
current_comparison_df = pd.DataFrame.from_dict(comparison_results, orient='index')

# Rename the index to 'Organ' for better readability
current_comparison_df.index.name = 'Organ'

# Store the current comparison results in the persistent dictionary, keyed by the selected model
selected_model = model_selector.value # Assuming model_selector is available from previous cells
all_comparison_results[selected_model] = current_comparison_df

print(f"Comparison results for '{selected_model}':")
# Display the current DataFrame
display(current_comparison_df)

# Analyze the results for the current model
print(f"\nAnalysis of Segmentation Metrics for '{selected_model}':")
print("-" * 30)

if not current_comparison_df.empty:
    # Find organs with highest/lowest Dice and Jaccard scores
    highest_dice_organ = current_comparison_df['dice'].idxmax()
    lowest_dice_organ = current_comparison_df['dice'].idxmin()
    highest_jaccard_organ = current_comparison_df['jaccard'].idxmax()
    lowest_jaccard_organ = current_comparison_df['jaccard'].idxmin()

    # Find organs with highest/lowest Hausdorff distances
    # Exclude infinite Hausdorff distances for min calculation if any
    finite_hausdorff = current_comparison_df[current_comparison_df['hausdorff'] != np.inf]['hausdorff']
    highest_hausdorff_organ = current_comparison_df['hausdorff'].idxmax()

    if not finite_hausdorff.empty:
        lowest_hausdorff_organ = finite_hausdorff.idxmin()
    else:
        lowest_hausdorff_organ = "N/A (All Hausdorff distances are infinite)"


    print(f"Overall Performance (based on selected organs):")
    print(f"- Dice Coefficient: Higher values indicate better overlap.")
    print(f"  Highest Dice: '{highest_dice_organ}' ({current_comparison_df.loc[highest_dice_organ, 'dice']:.4f})")
    print(f"  Lowest Dice: '{lowest_dice_organ}' ({current_comparison_df.loc[lowest_dice_organ, 'dice']:.4f})")
    print(f"- Jaccard Index (IoU): Higher values indicate better overlap.")
    print(f"  Highest Jaccard: '{highest_jaccard_organ}' ({current_comparison_df.loc[highest_jaccard_organ, 'jaccard']:.4f})")
    print(f"  Lowest Jaccard: '{lowest_jaccard_organ}' ({current_comparison_df.loc[lowest_jaccard_organ, 'jaccard']:.4f})")
    print(f"- Hausdorff Distance: Lower values indicate better boundary agreement.")
    print(f"  Highest Hausdorff: '{highest_hausdorff_organ}' ({current_comparison_df.loc[highest_hausdorff_organ, 'hausdorff']:.4f})")
    if lowest_hausdorff_organ != "N/A (All Hausdorff distances are infinite)":
         print(f"  Lowest Hausdorff: '{lowest_hausdorff_organ}' ({current_comparison_df.loc[lowest_hausdorff_organ, 'hausdorff']:.4f})")
    else:
        print(f"  Lowest Hausdorff: {lowest_hausdorff_organ}")

else:
    print("No comparison results to analyze.")

# Optionally, display all stored results
print("\nAll Stored Comparison Results:")
print("=" * 30)
for model_name, results_df in all_comparison_results.items():
    print(f"\nResults for Model: {model_name}")
    display(results_df)

Comparison results for 'totalSegmentator':


,dice,jaccard,hausdorff
Organ,,,
lung_lower_lobe_left,0.990787,0.981742,7.141428
lung_lower_lobe_right,0.990334,0.980853,2.828427
lung_middle_lobe_right,0.945697,0.896989,41.785165
lung_upper_lobe_left,0.994983,0.990015,3.605551
lung_upper_lobe_right,0.982896,0.966367,32.710854



Analysis of Segmentation Metrics for 'totalSegmentator':
------------------------------
Overall Performance (based on selected organs):
- Dice Coefficient: Higher values indicate better overlap.
  Highest Dice: 'lung_upper_lobe_left' (0.9950)
  Lowest Dice: 'lung_middle_lobe_right' (0.9457)
- Jaccard Index (IoU): Higher values indicate better overlap.
  Highest Jaccard: 'lung_upper_lobe_left' (0.9900)
  Lowest Jaccard: 'lung_middle_lobe_right' (0.8970)
- Hausdorff Distance: Lower values indicate better boundary agreement.
  Highest Hausdorff: 'lung_middle_lobe_right' (41.7852)
  Lowest Hausdorff: 'lung_lower_lobe_right' (2.8284)

All Stored Comparison Results:

Results for Model: totalSegmentator


,dice,jaccard,hausdorff
Organ,,,
lung_lower_lobe_left,0.990787,0.981742,7.141428
lung_lower_lobe_right,0.990334,0.980853,2.828427
lung_middle_lobe_right,0.945697,0.896989,41.785165
lung_upper_lobe_left,0.994983,0.990015,3.605551
lung_upper_lobe_right,0.982896,0.966367,32.710854
